Notes (swahl 7/13):

Had to change urllib.request to urllib2.urlopen for python 2.7

Wikipedia searches use underscores between words, spaces don't work.

There is an obvious problem with names that will lead to dissambiguation pages (e.g. Drake -> Drake_(rapper) ). It would probably be easier to use a data source more specific to artists.


In [1]:
import anyjson as json
import requests
from bs4 import BeautifulSoup
#import scalpyr
import urllib
from json import load 
import pandas as pd
from pandas import DataFrame
import numpy as np
import codecs
#from urllib import request
from urllib2 import urlopen # python 2.7 version
#import urlopen
from json import load
#import sqlite3
#import csv
#import codecs
#import cStringIO
#import sys
import hashlib

In [2]:
import wikipedia as wiki
def ambiguous(page_name):
    try: 
        wiki.page(page_name)
    except:
        return True
    else:
        return False

ambiguous('Drake')

True

In [3]:
#ArtistList = DataFrame(['drake_(rapper)', 'lady_gaga'],columns=['artist_name'])

#ArtistList = DataFrame.from_csv('../mode/mode-report-run-707064319e01.csv', sep='\t',index_col=False)
ArtistList = pd.DataFrame.from_csv('../billboard/billboard_artist_names_2010_15.csv',index_col=None)
DailyStats = pd.DataFrame()
ArtistList['name_length'] = 0
for i in range(len(ArtistList)):
    ArtistList['name_length'][i] = len(ArtistList['artist_name_underscore'][i])
ArtistList.head()

,artist_name,artist_name_clean,artist_name_underscore,artist_name_query,artist_name_dash,name_length
0,Ke$ha,Ke$ha,ke$ha,ke$ha,ke$ha,0
1,Eminem,Eminem,eminem,eminem,eminem,0
2,Katy Perry,Katy Perry,katy_perry,katy+perry,katy-perry,0
3,Rihanna,Rihanna,rihanna,rihanna,rihanna,0
4,Bruno Mars,Bruno Mars,bruno_mars,bruno+mars,bruno-mars,0


In [ ]:
#ArtistList.iloc[:5].artist_name_dash.apply(ambiguous)
ArtistList.ambiguous = ArtistList.artist_name_dash.apply(ambiguous)

In [ ]:
ArtistList.head()

In [4]:
import re
def queryName(name):
    """
    Use regular expressions to change an artist name string into
    a valid wikipedia search string.
    """
    s = name
    if ', The' in s:
        s = 'The ' + s[:-5]
    r = re.compile(r"\s+")
    s = r.sub("_", s) # "a\nb\nc
    return s.lower()

def cleanName(name):
    """
    Use regular expressions to change an artist name string into
    a valid wikipedia search string.
    """
    s = name
    if ', The' in s:
        s = 'The ' + s[:-5]
    #r = re.compile(r"\s+")
    #s = r.sub("_", s) # "a\nb\nc
    return s
  
cleanName('Black Keys, The')    

'The Black Keys'

In [30]:
#ArtistList['search_term'] = ArtistList.artist_name.apply(queryName)
#ArtistList['artist_name_clean'] = ArtistList.artist_name.apply(cleanName)
#ArtistList.head()

In [12]:
#fullList = ArtistList
#ArtistList = ArtistList.iloc[0:5]
#ArtistList = fullList[44:]

In [5]:
# build up a list of month year tuples
year_months = [(str(y),"%02d" % m)  for y in range(2010,2015) for m in range(1,13)]
year_months += [(str(2015),"%02d" % m) for m in range(1,8)]
#year_months

In [6]:
reader = codecs.getreader("utf-8")
baseurl = "http://stats.grok.se/json/"
lang = 'en'
#year = '2015'
#month = '05'
DailyStats = DataFrame()
#for a in range(0, len(ArtistList) -1):
n = len(ArtistList)
for a in ArtistList.index.values[:5]:
#for a in ArtistList.index.values[41:100]:
    print a+1,'/',n
    for year,month in year_months:
        try:
            url= baseurl+lang+'/'+year+month+'/'
            #search_term = ArtistList['artist_name'][a]
            search_term = ArtistList['artist_name_underscore'][a]
            url += search_term
            #print url
            #response = request.urlopen(url)
            response = urlopen(url)
            json_obj = load(reader(response))
            d = json_obj['daily_views']
            s  = pd.Series(d,index=d.keys())
            df = pd.DataFrame(s)
            df.columns = ['PageViews']
            df['Artist'] = ArtistList['artist_name'][a]
            df['search_term'] = search_term
            df['Language'] = lang
            df['url'] = url
            DailyStats = DailyStats.append(df)

        except:
            print (ArtistList['artist_name'][a] )

1 / 631
http://stats.grok.se/json/en/201001/ke$ha
http://stats.grok.se/json/en/201002/ke$ha
http://stats.grok.se/json/en/201003/ke$ha
http://stats.grok.se/json/en/201004/ke$ha
http://stats.grok.se/json/en/201005/ke$ha
http://stats.grok.se/json/en/201006/ke$ha
http://stats.grok.se/json/en/201007/ke$ha
http://stats.grok.se/json/en/201008/ke$ha
http://stats.grok.se/json/en/201009/ke$ha
http://stats.grok.se/json/en/201010/ke$ha
http://stats.grok.se/json/en/201011/ke$ha
http://stats.grok.se/json/en/201012/ke$ha
http://stats.grok.se/json/en/201101/ke$ha
http://stats.grok.se/json/en/201102/ke$ha
http://stats.grok.se/json/en/201103/ke$ha
http://stats.grok.se/json/en/201104/ke$ha
http://stats.grok.se/json/en/201105/ke$ha
http://stats.grok.se/json/en/201106/ke$ha
http://stats.grok.se/json/en/201107/ke$ha
http://stats.grok.se/json/en/201108/ke$ha
http://stats.grok.se/json/en/201109/ke$ha
http://stats.grok.se/json/en/201110/ke$ha
http://stats.grok.se/json/en/201111/ke$ha
http://stats.grok.se/json/

In [15]:
#print url
print DailyStats.count()
DailyStats.tail()

PageViews      1209438
Artist         1209438
search_term    1209438
Language       1209438
dtype: int64


,PageViews,Artist,search_term,Language
2015-07-15,385,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-12,395,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-07,456,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-10,410,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-11,326,LunchMoney Lewis,lunchmoney_lewis,en


In [16]:
#DailyStats[DailyStats.Artist == 'Eminem'].sort()
#DailyStats0_44 = DailyStats
#DailyStats45_end = DailyStats

In [18]:
#DailyStats_all = DailyStats0_44.append(DailyStats45_end)

In [19]:
# why are all of the entries 31?
#g = DailyStats.groupby('Artist')
g = DailyStats_all.groupby('Artist')


In [33]:
#DailyStats_all['date'] = DailyStats_all.index

In [32]:
#DailyStats_all

In [34]:
DailyStats_all = DailyStats_all.sort(['Artist','date'])

In [36]:
DailyStats_all = DailyStats_all[['date','Artist','search_term','Language','PageViews']]

In [37]:
DailyStats_all.to_csv('DailyWikiStats_from_billboard_2010_15.csv',index=False)

In [42]:
# correct dates
DailyStats = pd.DataFrame.from_csv('DailyWikiStats_from_billboard_2010_15.csv',parse_dates=True)

DailyStats['date'] = DailyStats.index

#DailyStats['date'] = pd.to_datetime(DailyStats.date)


In [52]:
import datetime as dt
def convertDate(s):
    #fmt = '%m/%d/%y'
    fmt = '%Y-%m-%d'

    try:
        return dt.datetime.strptime(s, fmt)
    except:
        return None
    
DailyStats['date'] = DailyStats.date.apply(convertDate)

In [54]:
DailyStats['artist_name_clean'] = DailyStats.Artist.apply(cleanName)

DailyStats = DailyStats[['date','search_term','Artist','artist_name_clean','Language','PageViews']]

DailyStats.to_csv('DailyWikiStats_from_billboard_2010_15.csv',index=False)

In [12]:
billboard = pd.DataFrame.from_csv('../billboard/BillboardData_mod_2000.csv')
billboard10_15 = billboard[billboard.year > 2009]
TrackList = np.unique(billboard10_15.track)
AlbumList = np.unique(billboard10_15.album)
TrackList
billboard10_15['artist_name_clean'] = billboard10_15.artist_standardized.apply(cleanName)
billboard10_15['track_underscore'] = billboard10_15.track.apply(queryName)
billboard10_15['album_underscore'] = billboard10_15.album.apply(queryName)

/Users/swahl/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/swahl/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
billboard10_15['track_ambiguous'] = billboard10_15.track_underscore.apply(ambiguous)
billboard10_15['album_ambiguous'] = billboard10_15.album_underscore.apply(ambiguous)

In [18]:
billboard10_15[['artist_name_clean','track','track_underscore','album'] ]

,artist_name_clean,track,track_underscore,album
id,,,,
22903,Ke$ha,TiK ToK,tik_tok,Animal
12718,Eminem,Love The Way You Lie,love_the_way_you_lie,Recovery
22752,Katy Perry,California Gurls,california_gurls,Teenage Dream (Deluxe Edition)
32403,Rihanna,Rude Boy,rude_boy,Rated R
6466,Bruno Mars,Just The Way You Are,just_the_way_you_are,Doo-Wops & Hooligans
22753,Katy Perry,Firework,firework,Teenage Dream (Deluxe Edition)
38275,Usher,OMG,omg,Raymond v Raymond
13478,Far East Movement,Like A G6,like_a_g6,Free Wired
22754,Katy Perry,Teenage Dream,teenage_dream,Teenage Dream


In [17]:
reader = codecs.getreader("utf-8")
baseurl = "http://stats.grok.se/json/"
lang = 'en'
#year = '2015'
#month = '05'
DailyStats = DataFrame()
#for a in range(0, len(ArtistList) -1):
n = len(billboard10_15)
for i,a in enumerate(billboard10_15.index.values[:2]):
    print i+1,'/',n
    adf = DataFrame()
    for year,month in year_months:
        try:
            url= baseurl+lang+'/'+year+month+'/'
            search_term = billboard10_15['track_underscore'][a]
            url += search_term
            print url
            #response = request.urlopen(url)
            response = urlopen(url)
            json_obj = load(reader(response))
            d = json_obj['daily_views']
            s  = pd.Series(d,index=d.keys())
            df = pd.DataFrame(s)
            df.columns = ['PageViews']
            df['Artist'] = billboard10_15['artist_name_clean'][a]
            df['track'] = billboard10_15['track'][a]
            df['search_term'] = search_term
            df['Language'] = lang
            df['url'] = url
            adf = adf.append(df)

        except:
            print (billboard10_15[['artist_name_clean','track']][a] )
            
    DailyStats = DailyStats.append(adf)
    
DailyStats.head()

1 / 2300
http://stats.grok.se/json/en/201001/tik_tok
http://stats.grok.se/json/en/201002/tik_tok
http://stats.grok.se/json/en/201003/tik_tok
http://stats.grok.se/json/en/201004/tik_tok
http://stats.grok.se/json/en/201005/tik_tok
http://stats.grok.se/json/en/201006/tik_tok
http://stats.grok.se/json/en/201007/tik_tok
http://stats.grok.se/json/en/201008/tik_tok
http://stats.grok.se/json/en/201009/tik_tok
http://stats.grok.se/json/en/201010/tik_tok
http://stats.grok.se/json/en/201011/tik_tok
http://stats.grok.se/json/en/201012/tik_tok
http://stats.grok.se/json/en/201101/tik_tok
http://stats.grok.se/json/en/201102/tik_tok
http://stats.grok.se/json/en/201103/tik_tok
http://stats.grok.se/json/en/201104/tik_tok
http://stats.grok.se/json/en/201105/tik_tok
http://stats.grok.se/json/en/201106/tik_tok
http://stats.grok.se/json/en/201107/tik_tok
http://stats.grok.se/json/en/201108/tik_tok
http://stats.grok.se/json/en/201109/tik_tok
http://stats.grok.se/json/en/201110/tik_tok
http://stats.grok.se/js

,PageViews,Artist,track,search_term,Language,url
2010-01-09,1390,Ke$ha,TiK ToK,tik_tok,en,http://stats.grok.se/json/en/201001/tik_tok
2010-01-04,1390,Ke$ha,TiK ToK,tik_tok,en,http://stats.grok.se/json/en/201001/tik_tok
2010-01-31,1212,Ke$ha,TiK ToK,tik_tok,en,http://stats.grok.se/json/en/201001/tik_tok
2010-01-30,1182,Ke$ha,TiK ToK,tik_tok,en,http://stats.grok.se/json/en/201001/tik_tok
2010-01-11,1073,Ke$ha,TiK ToK,tik_tok,en,http://stats.grok.se/json/en/201001/tik_tok


In [19]:
DailyStats.tail()

,PageViews,Artist,track,search_term,Language,url
2015-07-01,447,Eminem,Love The Way You Lie,love_the_way_you_lie,en,http://stats.grok.se/json/en/201507/love_the_w...
2015-07-03,460,Eminem,Love The Way You Lie,love_the_way_you_lie,en,http://stats.grok.se/json/en/201507/love_the_w...
2015-07-02,517,Eminem,Love The Way You Lie,love_the_way_you_lie,en,http://stats.grok.se/json/en/201507/love_the_w...
2015-07-09,447,Eminem,Love The Way You Lie,love_the_way_you_lie,en,http://stats.grok.se/json/en/201507/love_the_w...
2015-07-08,429,Eminem,Love The Way You Lie,love_the_way_you_lie,en,http://stats.grok.se/json/en/201507/love_the_w...


In [19]:
billboard10_15['track'][22903]

'TiK ToK'

In [22]:
json_obj

{u'daily_views': {u'2015-07-01': 28,
  u'2015-07-02': 27,
  u'2015-07-03': 53,
  u'2015-07-04': 15,
  u'2015-07-05': 25,
  u'2015-07-06': 29,
  u'2015-07-07': 30,
  u'2015-07-08': 28,
  u'2015-07-09': 26,
  u'2015-07-10': 18,
  u'2015-07-11': 13,
  u'2015-07-12': 19,
  u'2015-07-13': 22,
  u'2015-07-14': 11,
  u'2015-07-15': 19,
  u'2015-07-16': 37,
  u'2015-07-17': 20,
  u'2015-07-18': 34,
  u'2015-07-19': 23,
  u'2015-07-20': 27,
  u'2015-07-21': 24,
  u'2015-07-22': 23,
  u'2015-07-23': 17},
 u'month': u'201507',
 u'project': u'en',
 u'rank': -1,
 u'title': u'Just'}